In [1]:
import pandas as pd

In [2]:
# Load dataset

from tdc.multi_pred import MTI

data = MTI(name = 'miRTarBase')
data = data.neg_sample(frac = 1)
split = data.get_split()

Found local copy...
Loading...
Done!


AttributeError: 'DataFrame' object has no attribute 'append'

In [3]:
template = """### Instructions: Answer the following question about miRNA protein interactions. 
### Context: MicroRNAs (miRNAs) are, small non-coding RNAs with 18–25 nucleotides, which are central regulators at the 
post-transcriptional level in both animals and plants. Perfect or near-perfect complementary binding of miRNAs and their 
target mRNA negatively regulates gene expression by accelerating mRNA degradation or suppressing mRNA translation.
### Question: Given the miRNA mature sequence and target amino acid sequence, predict whether 
(A) the miRNA and target do not interact (B) the miRNA and target interact
miRNA sequence: {}
Target amino acid sequence: {}
### Answer: {}"""


In [4]:
split["train"].head()

NameError: name 'split' is not defined

In [5]:
# process dataset

def formatting_prompts_func(examples):
    peptide = examples["Peptide"]
    mhc = examples["MHC"]
    outputs       = examples["Y"]
    texts = []
    for p,m,o in zip(peptide, mhc, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = template.format(p,m, "(B)" if o else "(A)")
        texts.append(text)
    return { "text" : texts, "peptide":peptide, "mhc":mhc, "outputs":outputs}

def formatting_prompts_func_test(examples):
    peptide = examples["Peptide"]
    mhc = examples["MHC"]
    outputs       = examples["Y"]
    texts = []
    for p,m,o in zip(peptide, mhc, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = template.format(p,m, "")
        texts.append(text)
    return { "text" : texts, "peptide":peptide, "mhc":mhc, "outputs":outputs}

In [6]:
train_dataset = formatting_prompts_func(split["train"])
valid_dataset = formatting_prompts_func(split["valid"])
test_dataset = formatting_prompts_func_test(split["test"])

In [7]:
len(train_dataset["text"]), len(valid_dataset["text"]), len(test_dataset["text"])

(130190, 18598, 37197)

In [8]:
import pickle
import os


def dump_file(obj, path, file_name):
    os.makedirs(path, exist_ok=True)
    file_name = os.path.join(path,file_name)
    with open(file_name, "wb") as f0:
        pickle.dump(obj,f0)

In [9]:
dump_file(train_dataset, "processed", "train.pkl")
dump_file(valid_dataset, "processed", "valid.pkl")
dump_file(test_dataset, "processed", "test.pkl")